# Build Retrival Augmented Generation System using Amazon EMR Spark Distributed Processing and OpeSearch Vector Database 

In this notebook, we demonstrage users how to build a Retrival Augmented Generation (RAG) system using Mini-lm embeding model and `Llama2` Text Generation LLM.

We're leveraging OpenSearch Vector DB to store and retrive document embeddings.

## Connect to an Existing EMR Cluster

In [2]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --verify-certificate False --cluster-id j-NVBGPRZK5XVX --auth-type None --language python  

Successfully read emr cluster(j-NVBGPRZK5XVX) details
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1698298206599_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-NVBGPRZK5XVX", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


## Upload Files from Local to S3

In [16]:
%%local
import os
import glob
import boto3
import pikepdf
from tqdm import tqdm

In [17]:
%%local
pdf_files = glob.glob("./AWSGuides/*.pdf")
print(f"Total files ---> {len(pdf_files)}")

s3_client = boto3.client("s3")

for pdf_file in tqdm(pdf_files, total=len(pdf_files)):
    _filename = os.path.basename(pdf_file)
    _filename = _filename.replace(",", "").replace(" ", "-")
    s3_client.upload_file(
        pdf_file, 
        "sagemaker-us-east-1-280318901237", 
        os.path.join("Lab03", "raw-pdfs", _filename)
    )

Total files ---> 5


100%|██████████| 5/5 [00:01<00:00,  3.01it/s]


## Lets Convert PDF into Text

In [18]:
import os
import boto3
import json
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
import io

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
SRC_BUCKET_NAME = "sagemaker-us-east-1-280318901237"
SRC_FILE_PREFIX = "Lab03/raw-pdfs"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def list_files_in_s3_bucket_prefix(bucket_name, prefix):
    
    s3 = boto3.client('s3')

    # Paginate through the objects in the specified bucket and prefix, and collect all keys (file paths)
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    file_paths = []
    for page in page_iterator:
        if "Contents" in page:
            for obj in page["Contents"]:
                if os.path.basename(obj["Key"]):
                    file_paths.append(obj["Key"])

    return file_paths

all_pdf_files = list_files_in_s3_bucket_prefix(
    bucket_name=SRC_BUCKET_NAME, 
    prefix=SRC_FILE_PREFIX
)
print(f"Found {len(all_pdf_files)} files!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found 5 files!

In [21]:
all_pdf_files = [(SRC_BUCKET_NAME, fpath) for fpath in all_pdf_files]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
pdfs_rdd = spark.sparkContext.parallelize(all_pdf_files)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def load_pdf_from_s3(row):
    """
    Load a PDF file from an S3 bucket into memory.
    """
    try:
        src_bucket_name, src_file_key = row 
        s3 = boto3.client('s3')
        pdf_file = io.BytesIO()
        s3.download_fileobj(src_bucket_name, src_file_key, pdf_file)
        pdf_file.seek(0)
        pdf_reader = PdfReader(pdf_file)
        return (src_file_key, pdf_reader, len(pdf_reader.pages))
    
    except Exception as e:    
        return (os.path.basename(src_file_key), str(e))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
pdfs_in_memory = pdfs_rdd.map(load_pdf_from_s3).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
pdfs_in_memory

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Lab03/raw-pdfs/AWSLambdaDeveloperGuide.pdf', <PyPDF2._reader.PdfReader object at 0x7f7608188040>, 81), ('Lab03/raw-pdfs/AmazonSageMakerDeveloperGuide.pdf', <PyPDF2._reader.PdfReader object at 0x7f7608188d60>, 1055), ('Lab03/raw-pdfs/EC2DeveloperGuide.pdf', <PyPDF2._reader.PdfReader object at 0x7f7623eae4f0>, 698), ('Lab03/raw-pdfs/EMRManagementGuide.pdf', <PyPDF2._reader.PdfReader object at 0x7f761c658040>, 918), ('Lab03/raw-pdfs/S3DeveloperGuide.pdf', <PyPDF2._reader.PdfReader object at 0x7f761bf9e040>, 553)]

In [26]:
sum([pg_num for _, _, pg_num in pdfs_in_memory])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3305

In [27]:
class CustomDocument:
    def __init__(self, text, path, number):
        self.page_content = text
        self.metadata = {
            'source': path, 
            'page': number  
        }

    def __repr__(self):
        # This method is for representing the object in a way that’s clear to a human (also can be used for debugging)
        return f"Document(page_content='{self.page_content}', metadata={self.metadata})"

    # Optionally, if you need a string representation of the instance that is more user-friendly, 
    # you can implement the __str__ method
    def __str__(self):
        return f"Page Content: {self.page_content}\nSource: {self.metadata['source']}\nPage Number: {self.metadata['page']}"
    
def extract_text_from_pdf_reader(row):
    """ 
    Extract text from a page of the document 
    """
    try:
        doc_path, page_num = row
        page_text = global_pdfs_in_mem_dict[doc_path].pages[page_num].extract_text()
        return page_text, doc_path, page_num
    except Exception as e:
        return str(e), doc_path, page_num

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
global_pdfs_in_mem_dict = {_key: pdf_reader for _key, pdf_reader, _ in pdfs_in_memory}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
docs_instances = []
for (file_src, _, page_count) in pdfs_in_memory:
    for pg_num in range(page_count):
        docs_instances.append((file_src, pg_num))
print(f"Created {len(docs_instances)} parallel instances to process!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Created 3305 parallel instances to process!

In [30]:
docs_instances_rdd = spark.sparkContext.parallelize(docs_instances)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
documents = docs_instances_rdd.map(extract_text_from_pdf_reader).collect()
documents_custom = [
    CustomDocument(text=text, path=doc_source, number=page_num) 
    for text, doc_source, page_num in documents
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
documents_custom[121]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Document(page_content='Amazon SageMaker Developer Guide
Step 7.2: Validate a Model Deployed with Batch Transform
3. Get the Amazon SageMaker runtime client, which provides the invoke_endpoint  method.
runtime_client = boto3.client('runtime.sagemaker')
4. Get inferences from the ﬁrst 10 examples in the test dataset by calling invoke_endpoint .
with open('test_data', 'r') as f:
    
    for i in range(0,10):
        single_test = f.readline()
        response = runtime_client.invoke_endpoint(EndpointName = endpoint_name,
                                         ContentType = 'text/csv',
                                         Body = single_test)
        result = response['Body'].read().decode('ascii')
        print('Predicted label is {}.'.format(result))
5. To see if the model is making accurate predictions, check the output from this step against the
numbers you plotted in the previous step.
You have now trained, deployed, and validated your ﬁrst model in Amazon SageMaker.
Next Step
S

In [69]:
global_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
docs = global_text_splitter.split_documents(documents_custom)
print(f"Total number of docs after split {len(docs)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of docs after split 8841

In [71]:
print(docs[2695].page_content)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Amazon SageMaker Developer Guide
Amazon SageMaker Service
TransformInput
Service: Amazon SageMaker Service
Describes the input source of a transform job and the way the transform job consumes it.
Contents
CompressionType
If your transform data is compressed, specify the compression type. Amazon SageMaker
automatically decompresses the data for the transform job accordingly. The default value is None .
Type: String
Valid Values: None | Gzip
Required: No
ContentType
The multipurpose internet mail extension (MIME) type of the data. Amazon SageMaker uses the
MIME type with each http call to transfer data to the transform job.
Type: String
Length Constraints: Maximum length of 256.
Pattern: .*
Required: No
DataSource
Describes the location of the channel data, which is, the S3 location of the input data that the model
can consume.
Type: TransformDataSource (p. 1023 ) object
Required: Yes
SplitType

In [72]:
def generate_embeddings(input_text_sample):
    
    assert isinstance(input_text_sample, str), f"Input must be a single string but found " 
    
    lambda_client = boto3.client('lambda', region_name='us-east-1') 

    # Prepare the data to send to the Lambda function
    data = {
        "input": input_text_sample
    }

    # Invoke the Lambda function
    response = lambda_client.invoke(
        FunctionName="invoke-inference",
        InvocationType="RequestResponse",
        Payload=json.dumps(data)
    )

    # Decode and load the response payload
    response_payload = json.loads(response['Payload'].read().decode("utf-8"))

    # Extract status and embeddings from the response
    status_code, embeddings = int(response_payload['statusCode']), json.loads(response_payload['body'])

    return status_code, embeddings
    
class EmbeddingsGenerator:
    
    @staticmethod
    def embed_documents(input_text, normalize=True):
        """
        Generate embeddings for the provided text, invoking a Lambda function.
        """
        assert isinstance(input_text, list), "Input type must me list to embed_documents function"
        
        input_text_rdd = spark.sparkContext.parallelize(input_text)
        
        embeddings_generated = input_text_rdd.map(generate_embeddings).collect()
        
        embedding_response = []
        for s_code, embeddings in embeddings_generated:
            if s_code == 200:
                embedding_response.append(embeddings)
            else:
                pass
        
        return embedding_response
    
    @staticmethod
    def embed_query(input_text):
        status_code, embedding = generate_embeddings(input_text)
        if status_code == 200:
            return embedding
        else: 
            None

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
response_code, sample_sentence_embedding = generate_embeddings(docs[1001].page_content)
print(f"Status {response_code}, Embedding size of the document --->", len(sample_sentence_embedding))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Status 200, Embedding size of the document ---> 384

In [74]:
%%local
INDEX_NAME_OSE = "amz-data-dev-guides-index"

In [75]:
%%send_to_spark -i INDEX_NAME_OSE -t str -n INDEX_NAME_OSE

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'INDEX_NAME_OSE' as 'INDEX_NAME_OSE' to Spark kernel

In [76]:
import time
from langchain.vectorstores import OpenSearchVectorSearch


start = time.time()
docsearch = OpenSearchVectorSearch.from_documents(
    docs, 
    EmbeddingsGenerator, 
    opensearch_url="https://search-docsearchosdb-47fjsdj2ewsjpx4b34qz762kxy.us-east-1.es.amazonaws.com",
    bulk_size=len(docs),
    http_auth=("admin", "Admin123-"),
    index_name=INDEX_NAME_OSE,
    engine="faiss"
)

end = time.time()
print(f"Total Time for ingestion: {round(end - start, 2)} secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Time for ingestion: 929.39 secs

In [77]:
query = "What is a Training Job?"
sample_responses = docsearch.similarity_search(
    query, 
    k=5, 
    space_type="cosineSimilarity", 
    search_type="painless_scripting"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
sample_responses[0].page_content

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'jobs. To view a summary of the status of all of the training jobs that the hyperparameter tuning job\nlaunched, see Training job status counter.\n300'

In [80]:
%%local
!python3 -m pip install -q opensearch-py==2.3.2


[notice] A new release of pip available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [82]:
%%local
from langchain.vectorstores import OpenSearchVectorSearch


class EmbeddingGenerator:
    def __init__(self):
        self.lambda_client = boto3.client('lambda', region_name='us-east-1')
    
    def embed_query(self, input_text_sample):
        """Generate embeddings for the input text."""
        assert isinstance(input_text_sample, str), "Input must be a single string."
        
        # Prepare the data to send to the Lambda function.
        data = {"input": input_text_sample}

        # Invoke the Lambda function.
        response = self.lambda_client.invoke(
            FunctionName="invoke-inference",
            InvocationType="RequestResponse",
            Payload=json.dumps(data)
        )

        # Decode and load the response payload.
        response_payload = json.loads(response['Payload'].read().decode("utf-8"))

        # Extract status and embeddings from the response.
        status_code, embeddings = int(response_payload['statusCode']), json.loads(response_payload['body'])

        return embeddings


embedding_generator = EmbeddingGenerator()

docsearch = OpenSearchVectorSearch(
    index_name=INDEX_NAME_OSE,
    embedding_function=embedding_generator,
    opensearch_url="https://search-docsearchosdb-47fjsdj2ewsjpx4b34qz762kxy.us-east-1.es.amazonaws.com",
    http_auth=("admin", "Admin123-"),
    engine="faiss"
)

In [83]:
%%local
import re
import json
from typing import Dict
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.llms import SagemakerEndpoint
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA


class ContentHandler(LLMContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        
        pattern = r"(QUESTION:\n)(.*?)(\n\n)"
        
        match = re.search(pattern, prompt, re.DOTALL)
        
        # question_block = match.group(0)
        query_only = match.group(2)
        
        modified_prompt = re.sub(pattern, '', prompt, flags=re.DOTALL)
        
        body = {
            "inputs": [
                [
                     {
                         "role": "system", 
                         "content": modified_prompt
                     },
                    {
                        "role": "user", 
                        "content": query_only
                    },
                ]   
            ], 
            "parameters": model_kwargs
        }
        input_str = json.dumps(body)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        system_response = response_json[0]['generation']['content']
        return system_response.strip()

content_handler = ContentHandler()


llm_sm_ep = SagemakerEndpoint(
    endpoint_name="jumpstart-dft-meta-textgeneration-llama-2-7b-f", 
    region_name="us-east-1",
    model_kwargs={
        "max_new_tokens": 512, 
        "top_p": 1.0, 
        "temperature": 0.1, 
        "return_full_text": False
    },
    content_handler=content_handler,
    endpoint_kwargs={"CustomAttributes": 'accept_eula=true'}
)

In [86]:
%%local
from langchain import PromptTemplate

template = """
Answer the following QUESTION based on the CONTEXT
given. If you do not know the answer and the CONTEXT doesn't
contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""
prompt_template = PromptTemplate(
    template=template, 
    input_variables=['context', 'question']
)

llm_qa_smep_chain = RetrievalQA.from_chain_type(
    llm=llm_sm_ep,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={"k": 10, "space_type": "cosineSimilarity", "space_type": "painless_scripting"}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)


def pretty_print(chain_op):
    question = chain_op['query']
    
    response = chain_op['result']
    
    sources = "\n".join([f"-{src.metadata['source'].split('/')[-1]} (page: {src.metadata['page']})" for src in chain_op['source_documents']])
    
    stdout = f"""Question:\n> {question}\n\n================\nSystem:\n> {response}\n\n================\nSources:\n{sources}
    """
    print(stdout)

In [89]:
%%local
pretty_print(llm_qa_smep_chain("What is a SageMaker Training job and how do you run it?"))

Question:
> What is a SageMaker Training job and how do you run it?

System:
> Based on the provided context, a SageMaker training job is a process of using an algorithm to run a training job in Amazon SageMaker. To run a training job, you can use the Amazon SageMaker console, the low-level Amazon SageMaker API, or the Amazon SageMaker Python SDK.

Here are the general steps to run a training job:

1. Choose an algorithm that you want to use for training.
2. Provide information about the training job, such as the job name, IAM role, and S3 bucket where the output will be stored.
3. Launch the training job using the chosen algorithm.

The context provides more detailed information on how to run a training job using the Amazon SageMaker console, the low-level Amazon SageMaker API, or the Amazon SageMaker Python SDK.

Sources:
-AmazonSageMakerDeveloperGuide.pdf (page: 756)
-AmazonSageMakerDeveloperGuide.pdf (page: 428)
-AmazonSageMakerDeveloperGuide.pdf (page: 427)
-AmazonSageMakerDevelop

In [90]:
%%local
pretty_print(llm_qa_smep_chain("What types of instances are supported for Training Job?"))

Question:
> What types of instances are supported for Training Job?

System:
> Based on the context provided, the following instance types are supported for training jobs:

1. P2/P3 EC2 instances in single machine configurations.
2. ML instances (such as ml.p2.xlarge, ml.p2.8xlarge, ml.p2.16xlarge, ml.p3.2xlarge, ml.p3.8xlarge, and ml.p3.16xlarge) for both training and inference.

Note that the context does not mention any specific instance types for distributed training, so it is possible that only single machine configurations are supported for distributed training.

Sources:
-EMRManagementGuide.pdf (page: 13)
-AmazonSageMakerDeveloperGuide.pdf (page: 125)
-AmazonSageMakerDeveloperGuide.pdf (page: 119)
-AmazonSageMakerDeveloperGuide.pdf (page: 676)
-AmazonSageMakerDeveloperGuide.pdf (page: 210)
-AmazonSageMakerDeveloperGuide.pdf (page: 286)
-AmazonSageMakerDeveloperGuide.pdf (page: 420)
-AmazonSageMakerDeveloperGuide.pdf (page: 213)
-AmazonSageMakerDeveloperGuide.pdf (page: 756)
-Ama

In [91]:
%%local
pretty_print(llm_qa_smep_chain("How to install packages on EC2 instances using Command line?"))

Question:
> How to install packages on EC2 instances using Command line?

System:
> Based on the provided context, you can install packages on EC2 instances using the command line by following these steps:

1. Install the source code for the package you want to install using the `get_reference_source` command. For example, to download the source code for the `htop` package, you can run `get_reference_source -p htop`.
2. Install the package using the `pip` or `conda` command. For example, to install the `htop` package using `pip`, you can run `pip install htop`.
3. If you want to install multiple packages at once, you can use the `yum grouppinstall` command to install multiple packages and all their dependencies at once. For example, to install the `php` package and its dependencies, you can run `yum grouppinstall "Performance Tools"`.

Note: The `yum` command is used to install RPM packages, while `pip` and `conda` are used to install Python packages.

Also, you can use the `aws ec2 ru

In [96]:
%%local
pretty_print(llm_qa_smep_chain("How to Create a Training Job using Boto3 SDK?"))

Question:
> How to Create a Training Job using Boto3 SDK?

System:
> Based on the provided context, the answer to your question is:

I don't know.

The context does not provide enough information to answer your question about how to create a training job using the Boto3 SDK. The context only provides information on how to create a training job using the Amazon SageMaker Python SDK or the AWS SDK for Python (Boto 3). It does not provide any information on how to create a training job using the Boto3 SDK.

To create a training job using the Boto3 SDK, you will need to consult the AWS SDK for Python (Boto 3) documentation or seek additional information from a reliable source.

Sources:
-AmazonSageMakerDeveloperGuide.pdf (page: 31)
-AmazonSageMakerDeveloperGuide.pdf (page: 303)
-AmazonSageMakerDeveloperGuide.pdf (page: 2)
-AmazonSageMakerDeveloperGuide.pdf (page: 32)
-S3DeveloperGuide.pdf (page: 508)
-AmazonSageMakerDeveloperGuide.pdf (page: 854)
-AmazonSageMakerDeveloperGuide.pdf (page: 1

In [99]:
%%local
pretty_print(llm_qa_smep_chain("How can I deploy a model to SageMaker Hosting service?"))

Question:
> How can I deploy a model to SageMaker Hosting service?

System:
> To deploy a model to Amazon SageMaker hosting services, you can follow these steps:

1. Create a model in Amazon SageMaker by providing the location of the S3 bucket that contains the model artifacts and the registry path of the image that contains the inference code.
2. Create an endpoint configuration for an HTTPS endpoint by specifying the name of one or more models in production variants and the ML compute instances that you want Amazon SageMaker to launch to host each production variant.
3. Create an endpoint by sending a CreateEndpoint request to Amazon SageMaker, which launches the ML compute instances and deploys the model. Applications can then send requests for inference to this endpoint.

Note that you can also use the AWS SDK for Python (Boto 3) to deploy a model to Amazon SageMaker hosting services. The process is similar to the above steps, but you will need to use the CreateModel, CreateEndpoin

In [101]:
%%local
pretty_print(llm_qa_smep_chain("How do I validate a model using boto3 sdk and visualize results using matplotlib library?"))

Question:
> How do I validate a model using boto3 sdk and visualize results using matplotlib library?

System:
> To validate a model using the AWS SDK for Python (Boto 3) and visualize the results using the matplotlib library, you can follow these steps:

1. Download the test data from Amazon S3 using the `boto3.resource('s3')` object.
2. Use the `boto3.client('runtime.sagemaker')` object to invoke the endpoint and get the inferences for the first 10 examples in the test dataset.
3. Read the inferences from the response object and plot them using the matplotlib library.
4. Repeat steps 2 and 3 to get the inferences for the remaining examples in the test dataset.
5. Compare the predicted labels with the actual labels in the test dataset to validate the model.

Here is an example code snippet that demonstrates how to validate a model using the AWS SDK for Python (Boto 3) and visualize the results using the matplotlib library:
```python
import boto3
import matplotlib.pyplot as plt

# Down

In [102]:
%%local
pretty_print(llm_qa_smep_chain("How can I use the console to add a git repository to my SageMaker account?"))

Question:
> How can I use the console to add a git repository to my SageMaker account?

System:
> To add a Git repository to your Amazon SageMaker account using the console, follow these steps:

1. Open the Amazon SageMaker console at <https://console.aws.amazon.com/sagemaker/>.
2. Choose "Git repositories" from the left-hand menu.
3. Click on "Add repository" in the top right corner of the page.
4. Select "AWS CodeCommit" as the repository type.
5. Enter a name for the repository in the "Repository name" field. The name must be 1-63 characters long and can contain only letters (a-z), numbers (0-9), or dashes (-).
6. Choose "Use existing repository" or "Create a new repository" depending on whether you want to use an existing CodeCommit repository or create a new one.
7. If you choose "Use existing repository," select the repository from the list. If you choose "Create a new repository," enter a name for the repository and choose "Add repository."
8. If you want to associate the reposi